In [1]:
#import libraries required for analysis
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps

In [2]:
sql = '''
SELECT
    pct,
    CAST(month AS DATE) AS month,
    bnf_name,
    bnf_code,
    SUM(items) AS items,
    SUM(actual_cost) AS cost
FROM hscic.normalised_prescribing_standard presc
INNER JOIN hscic.practices pract ON presc.practice = pract.code
INNER JOIN
  hscic.ccgs AS ccg
ON
  presc.pct=ccg.code
WHERE
    ccg.org_type='CCG' AND
    pract.setting = 4 AND
    presc.bnf_code IN (
        SELECT DISTINCT(bnf_code)
        FROM ebmdatalab.measures.dmd_objs_with_form_route
        WHERE form_route IN ('pressurizedinhalation.inhalation', 'powderinhalation.inhalation')
        )
AND bnf_code LIKE "050%"
GROUP BY pct, month, bnf_name, bnf_code
ORDER BY pct, month
'''

df_inh_abx = bq.cached_read(sql, csv_path='df_inh_abx.csv')
df_inh_abx['month'] = df_inh_abx['month'].astype('datetime64[ns]')
df_inh_abx.head()

Downloading: 100%|██████████| 2052/2052 [00:00<00:00, 4042.49rows/s]


,pct,month,bnf_name,bnf_code,items,cost
0,00C,2010-12-01,Relenza_Disk 5mg & Diskhaler,0503040A0BBAAAA,1,15.10
1,00C,2011-01-01,Zanamivir_Pdr For Inh 5mg Disk + Dev,0503040A0AAAAAA,1,15.09
2,00C,2014-11-01,Tobi_Podhaler Pdr Inh Cap 28mg + Dev,0501040U0BCABAL,1,1650.93
3,00C,2015-01-01,Zanamivir_Pdr For Inh 5mg Disk + Dev,0503040A0AAAAAA,1,30.21
4,00C,2015-09-01,Tobramycin_Pdr Inh Cap 28mg + Dev,0501040U0AAALAL,1,1656.19


In [4]:
SELECT
     CAST(month AS DATE) AS month,
     practice AS practice_id,
     SUM(items) AS denominator
 FROM hscic.normalised_prescribing_standard
 WHERE bnf_code IN (   SELECT DISTINCT(bnf_code)   FROM measures.dmd_objs_with_form_route   WHERE form_route IN ('pressurizedinhalation.inhalation', 'powderinhalation.inhalation')   AND bnf_code LIKE '03%'    AND bnf_code NOT LIKE '0301011R0%'  )
 GROUP BY month, practice_id

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)